# CNNチュートリアル(keras)
MNISTを用いた簡単なチュートリアルについて説明します  
わからない部分や間違ってると感じた部分については質問してください  
なお、関連記事は大量にあるため、そちらの方も調べていただけるとより理解が増すと思います

## 実行環境
- macOS Sierra
- python3.6
- pip3

## 必要なライブラリのインポート

In [19]:
import glob
import os
import os.path
from keras.datasets import mnist
from keras.layers import MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Conv2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import np_utils

## モデルの実装
扱うモデル, データは, tutorial_tensorflow(padding='SAME'の場合)と同様です  
ただし, データはtensorflowではなくKeras経由でimportします  
tensorflowの記述と比べてシンプルに記述できることがわかると思います

In [32]:
TRAIN_DATA_SIZE = 100
TEST_DATA_SIZE = 100
BATCH_SIZE = 30
EPOCHS = 20
IMAGE_SIZE = 28

model = Sequential()

# 第一層
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 第二層
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 全結合層
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

# 作成したモデルを表示
model.summary()

# 誤差関数と最適化を加えてコンパイル
model.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
activation_37 (Activation)   (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
activation_38 (Activation)   (None, 11, 11, 64)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 1600)              0         
__________

## データの読み込み
この辺りはデータを読み込むことのできる形式に変換するためのコードになります  
reshapeの部分は, reshape(枚数, height, width, channel)と指定して変形を行います

In [33]:
 # MNISTの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 配列の整形と，色の範囲を0-255 -> 0-1に変換
X_train = X_train.reshape(60000, 28, 28, 1) / 255
X_test = X_test.reshape(10000, 28, 28, 1) / 255

# one-hot表現にする
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

## 学習, 予測
あとはデータを学習させ、テストデータに対して予測を行います  
学習のepoc数などに関しては統一していないためtensorflowの精度と差がありますが、ほとんど同じ精度で実装されます

In [34]:
hist = model.fit(X_train, y_train, batch_size=200, verbose=1, epochs=3, validation_split=0.1)

# 予測
score = model.evaluate(X_test, y_test, verbose=1)
print("")
print('test accuracy : ', score[1])

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 66s 1ms/step - loss: 0.2336 - acc: 0.9291 - val_loss: 0.0513 - val_acc: 0.9850
Epoch 2/3
54000/54000 [==============================] - 63s 1ms/step - loss: 0.0589 - acc: 0.9818 - val_loss: 0.0357 - val_acc: 0.9897
Epoch 3/3
10000/10000 [==============================] - 5s 459us/step

test accuracy :  0.9885
